<a href="https://colab.research.google.com/github/thuongvan23/XAI-LandCover/blob/kienNB/eurosat_random_forest_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q kaggle tifffile rasterio scikit-learn pandas numpy


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 83.2 MB/s eta 0:00:00


In [ ]:
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/Colab Notebooks/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d apollo2506/eurosat-dataset
!unzip -q eurosat-dataset.zip -d ./eurosat_data


Dataset URL: https://www.kaggle.com/datasets/apollo2506/eurosat-dataset
License(s): CC0-1.0
 97% 1.99G/2.04G [00:12<00:01, 40.1MB/s]
100% 2.04G/2.04G [00:12<00:00, 171MB/s] 


In [ ]:
import os
import pandas as pd
import numpy as np
import rasterio
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset


In [ ]:
class EuroSATAllBands(Dataset):
    def __init__(self, csv_file, root_dir):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.root_dir, row['Filename'])
        label = int(row['Label'])

        with rasterio.open(img_path) as src:
            img = src.read().astype(np.float32)
            img = img / 10000.0  # normalize

        return img, label


In [ ]:
def load_flattened_data(csv_path, root_dir):
    dataset = EuroSATAllBands(csv_path, root_dir)
    X, y = [], []
    for i in range(len(dataset)):
        img, label = dataset[i]
        X.append(img.flatten())
        y.append(label)
    return np.array(X), np.array(y)

base_path = "/content/eurosat_data/EuroSATallBands"

train_csv = os.path.join(base_path, "train.csv")
val_csv = os.path.join(base_path, "validation.csv")
test_csv = os.path.join(base_path, "test.csv")

X_train, y_train = load_flattened_data(train_csv, base_path)
X_val, y_val     = load_flattened_data(val_csv, base_path)
X_test, y_test   = load_flattened_data(test_csv, base_path)


In [ ]:
print("🎓 Training Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

train_acc = accuracy_score(y_train, rf.predict(X_train))
val_acc = accuracy_score(y_val, rf.predict(X_val))
test_acc = accuracy_score(y_test, rf.predict(X_test))

print(f"✅ Train Acc: {train_acc:.4f}")
print(f"✅ Val   Acc: {val_acc:.4f}")
print(f"✅ Test  Acc: {test_acc:.4f}")


🎓 Training Random Forest...
✅ Train Acc: 1.0000
✅ Val   Acc: 0.8206
✅ Test  Acc: 0.8383
